In [90]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from numba import njit, prange
from PIL import Image
from pathlib import Path

from src.utils import Picture
# Setting up I/O
input_dir = Path.cwd() / "data" / "raw"
output_dir = Path.cwd() / "data" / "output"

choose_pic = Picture.ISLAND
inp = Image.open(f"{input_dir}/{choose_pic.value}")
img = np.array(inp, dtype=np.float32)

In [68]:
# window = (
#     sliding_window_view(padded, (7, 7), axis=(0, 1))
#     .transpose(0, 1, 3, 4, 2)
# )
# window.shape

In [91]:
import numpy as np
from numba import njit, prange

# -----------------------
# 1. Параметры
# -----------------------

pad = 21
sigma = 8.0   # можно менять для силы blur

# -----------------------
# 2. Gaussian веса по радиусу
# -----------------------

r = np.arange(0, pad + 1, dtype=np.float32)
w = np.exp(-(r**2) / (2 * sigma**2)).astype(np.float32)
w /= w.sum()   # нормализация

# размер каждого кольца L∞
count = np.zeros(pad + 1, dtype=np.float32)
count[0] = 1.0
for i in range(1, pad + 1):
    count[i] = 8.0 * i

# -----------------------
# 3. Padding
# -----------------------

padded = np.pad(
    img.astype(np.float32),
    pad_width=((pad, pad), (pad, pad), (0, 0)),
    mode="reflect"
)

# -----------------------
# 4. Быстрый O(r) алгоритм
# -----------------------

@njit(parallel=True, fastmath=True)
def blur_ring_fast(img, w, count):
    pad = len(w) - 1
    H, W, C = img.shape
    out = np.zeros_like(img, dtype=np.float32)

    for y in prange(pad, H - pad):
        for x in range(pad, W - pad):

            for c in range(C):
                acc = 0.0

                for r in range(pad + 1):

                    k = w[r] / count[r]

                    if r == 0:
                        acc += img[y, x, c] * k
                    else:
                        # верх и низ
                        for j in range(-r, r + 1):
                            acc += img[y - r, x + j, c] * k
                            acc += img[y + r, x + j, c] * k

                        # лево и право (без углов)
                        for i in range(-r + 1, r):
                            acc += img[y + i, x - r, c] * k
                            acc += img[y + i, x + r, c] * k

                out[y, x, c] = acc

    return out


# -----------------------
# 5. Запуск
# -----------------------

nout = blur_ring_fast(padded, w, count)

# обрезаем padding
nout = nout[pad:-pad, pad:-pad]

# возвращаем в uint8
nout = np.clip(nout, 0, 255).astype(np.uint8)


In [86]:
nout = np.clip(nout, 0, 255).astype('uint8')

In [72]:
# output = np.einsum('hwijk,ij->hwk', window, kernel)

In [73]:
# output = np.clip(output, 0, 255).astype(np.uint8)

In [92]:
Image.fromarray(nout).save(f"{output_dir}/{choose_pic} - blured by numba baby.jpg")

In [75]:
# window = sliding_window_view(img, (3,3))
# window.shape